In [26]:
import os
import melee 
import numpy as np


In [27]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot'

In [ ]:
# console = melee.Console(is_dolphin=False,
#                         allow_old_version=False,
#                         path="./data/BTSSmash-Game_20200126T202356.slp"
#                         )
# console.connect()

# while True:
#     gamestate = console.step()
#     # step() returns None when the file ends
#     if gamestate is None:
#         break
#     print("Frame " + str(gamestate.frame))
#     for _, player in gamestate.players.items():
#         print("\t", player.stock, player.percent)

TypeError: Console.__init__() got an unexpected keyword argument 'is_dolphin'

In [40]:
import peppi_py as peppi
import numpy as np

def get_numpy_row(pre, post):
    def safe_extract(val):
        return val.to_numpy() # if val is not None else np.nan 

    features = []

    # --- Pre ---
    features += [
        safe_extract(pre.position.x),
        safe_extract(pre.position.y),
        safe_extract(pre.joystick.x),
        safe_extract(pre.joystick.y),
        safe_extract(pre.triggers),
        safe_extract(pre.buttons),
        safe_extract(pre.buttons_physical),
        safe_extract(pre.direction),
        safe_extract(pre.percent) if pre.percent else np.nan
    ]

    # --- Post ---
    features += [
        safe_extract(post.position.x),
        safe_extract(post.position.y),
        safe_extract(post.percent),
        safe_extract(post.shield),
        safe_extract(post.combo_count),
        safe_extract(post.state),
        safe_extract(post.airborne) if post.airborne else np.nan,
        safe_extract(post.stocks),
        safe_extract(post.last_hit_by),
        safe_extract(post.character),
        safe_extract(post.direction),
    ]

    return features

def slp_to_numpy(path, num_ports=2):
    game = peppi.read_slippi(path)
    rows = []

    for port in range(num_ports):
        frame = game.frames
        try:
            pre = frame.ports[port].leader.pre
            post = frame.ports[port].leader.post
            row = get_numpy_row(pre, post)
            rows.append(row)
        except Exception:
            # If frame/port is missing or broken, skip or pad
            rows.append([np.nan] * 20)

    arr = np.array(rows)
    # transpose to (12158, 2, 20)
    arr = arr.transpose(2, 0, 1)
    return arr.reshape(arr.shape[0], -1)

# Example usage
np_data = slp_to_numpy("./data/Stream-Game_20220828T225335.slp")
print("Shape:", np_data.shape)
print(np_data[:5])


None
Shape: (12158, 40)
[[-40.  32.   0.   0.   0. 256. 256.   1.   0. -40.  32.   0.  60.   0.
  322.   1.   4.   6.   7.   1.  40.  32.   0.   0.   0.   0.   0.  -1.
    0.  40.  32.   0.  60.   0. 322.   1.   4.   6.  18.  -1.]
 [-40.  32.   0.   0.   0. 256. 256.   1.   0. -40.  32.   0.  60.   0.
  322.   1.   4.   6.   7.   1.  40.  32.   0.   0.   0.   0.   0.  -1.
    0.  40.  32.   0.  60.   0. 322.   1.   4.   6.  18.  -1.]
 [-40.  32.   0.   0.   0. 256. 256.   1.   0. -40.  32.   0.  60.   0.
  322.   1.   4.   6.   7.   1.  40.  32.   0.   0.   0.   0.   0.  -1.
    0.  40.  32.   0.  60.   0. 322.   1.   4.   6.  18.  -1.]
 [-40.  32.   0.   0.   0. 256. 256.   1.   0. -40.  32.   0.  60.   0.
  322.   1.   4.   6.   7.   1.  40.  32.   0.   0.   0.   0.   0.  -1.
    0.  40.  32.   0.  60.   0. 322.   1.   4.   6.  18.  -1.]
 [-40.  32.   0.   0.   0. 256. 256.   1.   0. -40.  32.   0.  60.   0.
  322.   1.   4.   6.   7.   1.  40.  32.   0.   0.   0.   0.   0.  -1.
    

In [41]:
np.save("training_data.npy", np_data)